# Optilab tutorial: creating your own optimizer
This tutorial aims to explain more custom usecases for optilab. In this notebook you will learn how to:
- create your own objective function,
- create your own optimizer,
- perform optimization using optilab API and your custom components.

## Creating a custom objective function
Optilab allows you to create your own custom objective function by creating a class inheriting the `ObjectiveFunction` base class. Methods `__init__` and `__call__` need to be overloaded. Let's create an example objective function:

In [1]:
# import for typehint
from optilab.data_classes import FunctionMetadata, Point, Bounds
from optilab.functions import ObjectiveFunction


class CustomObjectiveFunction(ObjectiveFunction):
    def __init__(self, dim: int, exponent: float):
        # setting necessary metadata for the function
        super().__init__('custom_function', dim, {'exponent': exponent})
        self.exponent = exponent

    def __call__(self, point: Point) -> Point:
        # incrementing call counter
        super().__call__(point)

        # actual function value calculation
        return Point(
            x = point.x,
            y = sum([x_i ** self.metadata.hyperparameters['exponent'] for x_i in point.x]),
            is_evaluated = True
        )  

As you can see the preferred way of accessing the hyperparameters are through `self.metadata.hyperparameters`. Let's now evaluate an example point with the complex function:

In [2]:
example_point = Bounds(-1, 1).random_point(10)
custom_func = CustomObjectiveFunction(10, 3)
print(custom_func(example_point))

Point(x=array([ 0.75485221, -0.55744804, -0.51548595, -0.42401401,  0.28025724,
       -0.4802263 , -0.88308752,  0.38495085, -0.07605388,  0.4949106 ]), y=np.float64(-0.5558998516922915), is_evaluated=True)


By overloading the constructor of the function, the function's metadata is set. This includes the name, dimensionality, and other optional hyperparameters. The metadata is stored as a member of the class. Let's create an instance of the custom objective function and look at it's metadata:

In [3]:
custom_func.metadata

FunctionMetadata(name='custom_function', dim=10, hyperparameters={'exponent': 3})

## Creating a custom optimizer
Let's now create a custom optimizer class. Similarily to the objective function, to create a custom optimizer you need to create a class inheriting `Optimizer` class. `__init__` and `optimize` methods have to be overloaded. `run_optimization` method is implemented in the base class and must not be overloaded.

Let's create a placeholder custom optimizer:

In [4]:
from optilab.optimizers import Optimizer
from optilab.data_classes import PointList, Bounds


import numpy as np
from typing import Any, Dict, List, Tuple


class CustomOptimizer(Optimizer):    
    def __init__(
        self,
        population_size: int,
        custom_hyperparameter: float=0.8
    ) -> None:
        super().__init__(
            'custom_optimizer',
            population_size,
            {
                'custom_hyperparameter': custom_hyperparameter
            }
        )
    
    def optimize(
        self,
        function: ObjectiveFunction,
        bounds: Bounds,
        call_budget: int,
        tolerance: float,
        target: float = 0.0
    ) -> PointList:
        # this is where the logic of the optimizer must be places
        # as this is an example, random points will be returned
        population = bounds.random_point_list(self.metadata.population_size, function.metadata.dim)

        result_log = PointList([function(pt) for pt in population])

        # return PointList of all evaluated points
        return result_log


Let's now create an instance of this custom optimizer and perform 15 optimizations of our custom function.

In [5]:
optimizer = CustomOptimizer(10, 0.5)
run = optimizer.run_optimization(num_runs=15, function=custom_func, bounds=Bounds(-10, 10), call_budget=1e4, tolerance=1e-10, target=0.0)

Optimizing...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 8501.97run/s]
